### 학습 목표
1. BERT의 Embeddings모듈 동작을 이해하고 구현할 수 있다.
2. BERT의 Self-Attention을 활용한 Transformer 부분인 BertLayer모듈의 동작을 이해하고 구현할 수 있다.
3. BERT의 Pooler모듈의 동작을 이해하고 구현할 수 있다.

## 8.2.2 BERT_Base의 네트워크 설정 파일 읽기
- 먼저 BERT_Base에서 Transformer가 12단인 것과 특징량 벡터가 768차원인 것 등을 적은 weights폴더의 네트워크 설정 파일 bert_config.json을 읽어들입니다.
- 읽어들인 JSON파일의 사전형 변수에서 key 'hidden'값을 취하려면 config['hidden_size']로 적어야합니다. 이를 config.hidden_size로 기술하면 깔끔합니다.

In [2]:
# config.json에서 설정을 읽어들여 JSON 사전 변수를 오브젝트 변수로 변환
import json

config_file = './weights/bert_config.json'

# 파일을 열어 JSON으로 읽는다.
with open(config_file, 'r') as f:
    config = json.load(f)
config

{'attention_probs_dropout_prob': 0.1,
 'hidden_act': 'gelu',
 'hidden_dropout_prob': 0.1,
 'hidden_size': 768,
 'initializer_range': 0.02,
 'intermediate_size': 3072,
 'max_position_embeddings': 512,
 'num_attention_heads': 12,
 'num_hidden_layers': 12,
 'type_vocab_size': 2,
 'vocab_size': 30522}

In [3]:
# 사전 변수를 오브젝트 변수로
from attrdict import AttrDict

config = AttrDict(config)
config.hidden_size

768

## 8.2.3 BERT에 레이어 정규화 층 정의
- BERT 모델 구축의 사전 준비로 레이어 정규화 층의 클래스를 정의합니다. 7장에서 사용한 것 처럼 파이토치에도 레이어 정규화가 있습니다.
- 텐서플로와 파이토치에서는 레이어 정규화의 구현 방법이 약간 다릅니다. 텐서의 마지막 채널(즉 단어의 특징량 벡터 768차원)에 평균 0, 표준편차 1이 되도록 레이어 정규화를 수행합니다. 0으로 나누지 않도록 보조 항 엡실론을 넣는 방법은 파이토치와 텐서플로가 서로 다릅니다.
- 이번에 사용할 학습된 모델은 구글이 공개한 텐서플로의 학습 결과에 기반하여 텐서플로 버전의 레이어 정규화 층을 만듭니다.

In [4]:
# BERT용으로 레이어 정규화 층 정의
# 세부 구현을 텐서플로에 맞춘다.
import torch.nn as nn
import torch

class BertLayerNorm(nn.Module):
    '''레이어 정규화 층'''
    
    def __init__(self, hidden_size, eps=1e-12):
        super(BertLayerNorm, self).__init__()
        self.gamma = nn.Parameter(torch.ones(hidden_size)) # weight에 대한 것
        self.beta = nn.Parameter(torch.zeros(hidden_size)) # 바이어스에 대한 것
        self.variance_epsilon = eps
    
    def forward(self, x):
        u = x.mean(-1, keepdim=True)
        s = (x - u).pow(2).mean(-1, keepdim=True)
        x = (x - u) / torch.sqrt(s + self.variance_epsilon)
        return self.gamma * x + self.beta

## 8.2.4 Embedding 구현
### Transformer의 Embeddings 모듈과 두 가지 큰 차이점 존재
- 첫째, Positional Embedding(위치 정보를 벡터로 변환)의 표현 기법을 Transformer는 sin, cos으로 계산하지만 BERT는 표현 방법도 학습시킵니다. 학습 시키는 것은 단어의 위치 정보뿐이며 단어 벡터의 차원 정보는 부여하지 않습니다. 즉 첫 번째 단어의 768차원은 동일한 position_embeddings값이 저장 되고 두 번째 단어는 첫 번째 단어와는 다르지만 768차원 방향에 같은 position_embeddings값이 저장됩니다.
- 둘째, Sentence Embedding의 존재입니다. BERT는 두 문장을 입력합니다. 첫 번째 문장과 두 번째 문장을 구분하기 위한 Embedding를 준비합니다. Embeddings 모듈에서는 Token Embedding, Positional Embedding, Sentence Embedding에서 각각 구할 세 개의 텐서를 Transformer처럼 더하여 Embeddings 모듈의 출력으로 합니다. Embeddings모듈에 대한 입력 텐서는 (batch_size, seq_len)크기로 이루어진 문장의 단어 ID 나열인 변수 input_ids와 (batch_size, seq_len)의 각 단어가 첫 번째 문장인지 두 번째 문장인지 나타내는 문장 id인 변수 token_type_ids가 됩니다. 출력은 (batch_size, seq_len, hidden_size)의 텐서입니다. seq_len은 512이고 hidden_size는 768입니다.

In [5]:
# BERT의 Embeddings 모듈
class BertEmbeddings(nn.Module):
    '''문장의 단어 ID열과 첫 번째인지 두 번째 문장인지 정보를 내장 벡터로 변환'''
    
    def __init__(self, config):
        super(BertEmbeddings, self).__init__()
        
        # 세 개의 벡터 표현 내장
        
        # Token Embedding: 단어 ID를 단어 벡터로 변환
        # vocab_size = 30522로 BERT의 학습된 모델에 사용된 vocabulary 양
        # hidden_size = 768로 특징량 벡터의 길이는 768
        self.word_embeddings = nn.Embedding(
            config.vocab_size, config.hidden_size, padding_idx=0)
        
        # padding_idx = 0의 idx = 0 단어 벡터는 0으로 한다. BERT의 vocabulary의 idx=0은 [PAD]이다
        
        # Transformer Positional Embedding: 위치 정보 텐서를 벡터로 변환
        # Transformer의 경우는 sin, cos로 이루어진 고정 값이지만 BERT는 학습시킨다.
        # max_position_embeddings = 512로 문장 길이는 512단어
        self.position_embeddings = nn.Embedding(
            config.max_position_embeddings, config.hidden_size)
        
        # Sentence Embedding: 첫 번째, 두 번째 문장을 벡터로 변환
        # type_vocab_size = 2
        self.token_type_embeddings = nn.Embedding(
            config.type_vocab_size, config.hidden_size)
        
        # 작성한 레이어 정규화 층
        self.LayerNorm = BertLayerNorm(config.hidden_size, eps=1e-12)
        
        # 드롭아웃 'hidden_dropout_prob':0.1
        self.dropout = nn.Dropout(config.hidden_dropout_prob)
        
    def forward(self, input_ids, token_type_ids=None):
        '''
        input_ids: [batch_size, seq_len] 문장의 단어 ID 나열
        token_type_ids: [batch_size, seq_len] 각 단어가 첫 번째 문장인지 두 번째 문장인지 나타내는 id
        '''
        
        # 1. Token Embeddings
        # 단어 ID를 단어 벡터로 변환
        words_embeddings = self.word_embeddings(input_ids)
        
        # 2. Sentence Embeddings
        # token_type_ids가 없는 경우는 문장의 모든 단어를 첫 번째 문장으로 하여 0으로 설정
        # input_ids와 같은 크기릐 제로 텐서 작성
        if token_type_ids is None:
            token_type_ids = torch.zeros_like(input_ids)
        token_type_embeddings = self.token_type_embeddings(token_type_ids)
        
        # 3. Transformer Positional Embedding:
        # [0, 1, 2, ...]로 문장의 길이 만큼 숫자가 하나씩 올라간다.
        # [batch_size, seq_len]의 텐서 positional_ids 작성
        # positional_ids를 입력하여 position_embeddings 층에서 768차원의 텐서를 꺼낸다
        seq_length = input_ids.size(1) # 문장 길이
        position_ids = torch.arange(
            seq_length, dtype=torch.long, device=input_ids.device)
        position_ids = position_ids.unsqueeze(0).expand_as(input_ids)
        position_embeddings = self.position_embeddings(position_ids)
        
        # 세 개의 내장 텐서를 더한다. [batch_size, seq_len, hidden_size]
        embeddings = words_embeddings + position_embeddings + token_type_embeddings
        
        # 레이어 정규화와 드롭아웃 실행
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)
        
        return embeddings

## 8.2.5 BertLayer 모듈
- BertLayer는 Transformer부분에 해당.
- 서브 네트워크로서 Self-Attention을 계산하는 BertAttention과 Self-Attention의 출력을 처리하는 전결합 층인 BertIntermediate, 그리고 Self-Attention 출력과 BertIntermediate에서 처리한 특징량을 더하는 BertOutput 세 가지로 구성됩니다.
- BertLayer에 대한 입력은 Embedding 모듈의 출력 또는 앞단의 BertLayer에서의 출력이며 크기는 (batch_size, seq_len, hidden_size)입니다. - BertLayer구현에서 7장 Transformer와 두 가지 다른 점이 있습니다.
- 첫째, BertIntermediate 전결합 층 뒤의 활성화 함수에 GELU함수를 사용하는 점입니다. GELU는 기본적으로 RELU와 같은 형태의 함수입니다. 입력이 0이지만 ReLU출력이 거친(매끄러운 변화가 아니라 급격환 변화) 반면 GELU는 입력 0 근처의 출력이 매끄러운 형태입니다.
- 둘쩨, Attention이 Multi-Headed Self-Attention입니다. Transformer도 Multi-Headed Self-Attention이지만 7장에서는 이해를 돕기 위하여 단일 Self-Attention으로 구현하였습니다. Multi-Headed Self-Attention은 단순히 Self-Attention이 여러 개 있는것 뿐입니다.